<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVISProject-CAValli_Team/blob/main/Preprocessing_story2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading and preprocessing data

In [2]:
all_files = glob.glob("/content/drive/MyDrive/DV_project/story2/*.xlsx")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/DV_project/story2/early_leavers_by_sex_labour.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_absolute.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_percentage.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_age.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_education.xlsx']


## Education level: absolute value

We import the 36 sheets of the *pop_edu_sex_age_absolute* excel file (skipping useless rows/columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *education_abs_sheets*.

In [36]:
years = range(2009, 2023)
sheet_names = [f'Sheet {i}' for i in range(1, 37)]
file_path = '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_absolute.xlsx'
rows_to_skip = list(range(0, 10)) + [11, 12, 13, 44] + list(range(49, 58))
selected_columns = ['TIME'] + [str(year) for year in years]

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'education_abs_sheet1', ..., 'education_abs_sheet36'
    education_abs_sheets = {f'education_abs_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                            for i, sheet_name in enumerate(sheet_names, start = 1)}

We create 2 dictionaries linking each country to its short abbreviation (*country_abbreviations*) and viceversa (*country_abbreviations_reversed*).

In [25]:
country_abbreviations = {
    'European Union - 27 countries (from 2020)': 'EU',
    'Belgium': 'BEL',
    'Bulgaria': 'BGR',
    'Czechia': 'CZE',
    'Denmark': 'DNK',
    'Germany': 'DEU',
    'Estonia': 'EST',
    'Ireland': 'IRL',
    'Greece': 'GRC',
    'Spain': 'ESP',
    'France': 'FRA',
    'Croatia': 'HRV',
    'Italy': 'ITA',
    'Cyprus': 'CYP',
    'Latvia': 'LVA',
    'Lithuania': 'LTU',
    'Luxembourg': 'LUX',
    'Hungary': 'HUN',
    'Malta': 'MLT',
    'Netherlands': 'NLD',
    'Austria': 'AUT',
    'Poland': 'POL',
    'Portugal': 'PRT',
    'Romania': 'ROU',
    'Slovenia': 'SVN',
    'Slovakia': 'SVK',
    'Finland': 'FIN',
    'Sweden': 'SWE',
    'Iceland': 'ISL',
    'Norway': 'NOR',
    'Switzerland': 'CHE',
    'Montenegro': 'MNE',
    'North Macedonia': 'MKD',
    'Serbia': 'SRB',
    'Türkiye': 'TUR',
}

country_abbreviations_reversed = {value: key for key, value in country_abbreviations.items()}

We add a column *Abbreviation* to the 36 datasets.

In [5]:
for i in range(1, 37):
    education_abs_sheets[f'education_abs_sheet{i}'].rename(columns = {'TIME': 'Country'}, inplace = True)
    education_abs_sheets[f'education_abs_sheet{i}']['Abbreviation'] = education_abs_sheets[f'education_abs_sheet{i}']['Country'].map(country_abbreviations)

We sum data about 15-24 y.o. people and data about 25-29 y.o. people to have data coherent to what we used in story 1 (people aged between 15 and 29).

We create 3 different dataset, one for each educational level (low, medium and high).

In [6]:
low1 = education_abs_sheets['education_abs_sheet2']
low2 = education_abs_sheets['education_abs_sheet8']

medium1 = education_abs_sheets['education_abs_sheet3']
medium2 = education_abs_sheets['education_abs_sheet9']

high1 = education_abs_sheets['education_abs_sheet6']
high2 = education_abs_sheets['education_abs_sheet12']

In [7]:
low1.set_index(['Country', 'Abbreviation'], inplace = True)
low2.set_index(['Country', 'Abbreviation'], inplace = True)
low = low1.fillna(1000000).add(low2.fillna(1000000), fill_value = 0).round(1)
low = low.where(low < 1000000)
low1.reset_index(inplace = True)
low2.reset_index(inplace = True)
low.reset_index(inplace = True)

In [8]:
medium1.set_index(['Country', 'Abbreviation'], inplace = True)
medium2.set_index(['Country', 'Abbreviation'], inplace = True)
medium = medium1.fillna(1000000).add(medium2.fillna(1000000), fill_value = 0).round(1)
medium = medium.where(medium < 1000000)
medium1.reset_index(inplace = True)
medium2.reset_index(inplace = True)
medium.reset_index(inplace = True)

In [9]:
high1.set_index(['Country', 'Abbreviation'], inplace = True)
high2.set_index(['Country', 'Abbreviation'], inplace = True)
high = high1.fillna(1000000).add(high2.fillna(1000000), fill_value = 0).round(1)
high = high.where(high < 1000000)
high1.reset_index(inplace = True)
high2.reset_index(inplace = True)
high.reset_index(inplace = True)

### Stacked barchart

We create one dataset for each year to use for the stacked barchart (percentage).

In [10]:
def create_stacked_dataset(year):
    stacked = pd.DataFrame()

    low_year = low[['Country', 'Abbreviation', year]].rename(columns = {year: 'low'})
    medium_year = medium[['Country', 'Abbreviation', year]].rename(columns = {year: 'medium'})
    high_year = high[['Country', 'Abbreviation', year]].rename(columns = {year: 'high'})

    stacked = pd.merge(low_year, medium_year, on = ['Country', 'Abbreviation'])
    stacked = pd.merge(stacked, high_year, on = ['Country', 'Abbreviation'])
    stacked = stacked.dropna(subset = ['low', 'medium', 'high'])

    return stacked

for year in range(2009, 2023):
    stacked_year = create_stacked_dataset(str(year))
    stacked_year = stacked_year.sort_values(by = 'Country')
    stacked_year.to_csv(f"stacked{year}.csv", index = False, na_rep = 'nan')

for year in range(2009, 2023):
    stacked_year = create_stacked_dataset(str(year))
    stacked_year = stacked_year.sort_values(by = 'low', ascending = False)
    stacked_year.to_csv(f"stacked{year}_low.csv", index = False, na_rep = 'nan')

### Donut charts

We do the same for males.

In [10]:
low1_M = education_abs_sheets['education_abs_sheet14']
low2_M = education_abs_sheets['education_abs_sheet20']

medium1_M = education_abs_sheets['education_abs_sheet15']
medium2_M = education_abs_sheets['education_abs_sheet21']

high1_M = education_abs_sheets['education_abs_sheet18']
high2_M = education_abs_sheets['education_abs_sheet24']

In [11]:
low1_M.set_index(['Country', 'Abbreviation'], inplace = True)
low2_M.set_index(['Country', 'Abbreviation'], inplace = True)
low_M = low1_M.fillna(1000000).add(low2_M.fillna(1000000), fill_value = 0).round(1)
low_M = low_M.where(low_M < 1000000)
low1_M.reset_index(inplace = True)
low2_M.reset_index(inplace = True)
low_M.reset_index(inplace = True)

In [12]:
medium1_M.set_index(['Country', 'Abbreviation'], inplace = True)
medium2_M.set_index(['Country', 'Abbreviation'], inplace = True)
medium_M = medium1_M.fillna(1000000).add(medium2_M.fillna(1000000), fill_value = 0).round(1)
medium_M = medium_M.where(medium_M < 1000000)
medium1_M.reset_index(inplace = True)
medium2_M.reset_index(inplace = True)
medium_M.reset_index(inplace = True)

In [13]:
high1_M.set_index(['Country', 'Abbreviation'], inplace = True)
high2_M.set_index(['Country', 'Abbreviation'], inplace = True)
high_M = high1_M.fillna(1000000).add(high2_M.fillna(1000000), fill_value = 0).round(1)
high_M = high_M.where(high_M < 1000000)
high1_M.reset_index(inplace = True)
high2_M.reset_index(inplace = True)
high_M.reset_index(inplace = True)

We do the same for females.

In [14]:
low1_F = education_abs_sheets['education_abs_sheet26']
low2_F = education_abs_sheets['education_abs_sheet32']

medium1_F = education_abs_sheets['education_abs_sheet27']
medium2_F = education_abs_sheets['education_abs_sheet33']

high1_F = education_abs_sheets['education_abs_sheet30']
high2_F = education_abs_sheets['education_abs_sheet36']

In [15]:
low1_F.set_index(['Country', 'Abbreviation'], inplace = True)
low2_F.set_index(['Country', 'Abbreviation'], inplace = True)
low_F = low1_F.fillna(1000000).add(low2_F.fillna(1000000), fill_value = 0).round(1)
low_F = low_F.where(low_F < 1000000)
low1_F.reset_index(inplace = True)
low2_F.reset_index(inplace = True)
low_F.reset_index(inplace = True)

In [16]:
medium1_F.set_index(['Country', 'Abbreviation'], inplace = True)
medium2_F.set_index(['Country', 'Abbreviation'], inplace = True)
medium_F = medium1_F.fillna(1000000).add(medium2_F.fillna(1000000), fill_value = 0).round(1)
medium_F = medium_F.where(medium_F < 1000000)
medium1_F.reset_index(inplace = True)
medium2_F.reset_index(inplace = True)
medium_F.reset_index(inplace = True)

In [17]:
high1_F.set_index(['Country', 'Abbreviation'], inplace = True)
high2_F.set_index(['Country', 'Abbreviation'], inplace = True)
high_F = high1_F.fillna(1000000).add(high2_F.fillna(1000000), fill_value = 0).round(1)
high_F = high_F.where(high_F < 1000000)
high1_F.reset_index(inplace = True)
high2_F.reset_index(inplace = True)
high_F.reset_index(inplace = True)

We extract info on differences between M and F.

In [18]:
levels = ['low', 'medium', 'high']
genders = ['M', 'F']

sum_datasets = {}
difference_datasets = {}
percentage_datasets = {}

for level in levels:
    for gender in genders:

        male_dataframe = f"{level}_M"
        female_dataframe = f"{level}_F"
        sum_dataframe = f"{level}_sum"
        difference_dataframe = f"{level}_difference"
        percentage_dataframe = f"{level}_percentage"

        male_df = locals()[male_dataframe]
        female_df = locals()[female_dataframe]

        sum_df = male_df.copy()
        sum_df.iloc[:, 2:] = male_df.iloc[:, 2:] + female_df.iloc[:, 2:]
        sum_datasets[sum_dataframe] = sum_df

        difference_df = male_df.copy()
        difference_df.iloc[:, 2:] = male_df.iloc[:, 2:] - female_df.iloc[:, 2:]
        difference_datasets[difference_dataframe] = difference_df

        percentage_df = male_df.copy()
        percentage_df.iloc[:, 2:] = (difference_df.iloc[:, 2:] / sum_df.iloc[:, 2:]) * 100
        percentage_datasets[percentage_dataframe] = percentage_df

For each year (from 2009 to 2022) we retrieve the top 3 countries with highest percentual difference between M and F (for low, medium and high education level), considering just the ABSOLUTE VALUE of this percentage (NOT considering the sign).

In [21]:
def ordinal(n):
    suffix = 'th' if 11 <= n <= 13 else {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

# Function to convert the percentage column to numeric
def convert_to_numeric(df, column):
    df[column] = pd.to_numeric(df[column], errors = 'coerce')
    return df

level_datasets = {
    'low': {'M': low_M, 'F': low_F},
    'medium': {'M': medium_M, 'F': medium_F},
    'high': {'M': high_M, 'F': high_F}
}

for year in range(2009, 2023):
    print(f"\nIn the year {year}:")
    donut_data = []
    for level in levels:
        percentage_df = percentage_datasets[f"{level}_percentage"]

        # Reshape data using melt
        melted_df = pd.melt(percentage_df, id_vars = ['Country'], var_name = 'Year', value_name = f"{level}_difference")

        # Convert 'Year' and difference columns to numeric
        melted_df = convert_to_numeric(melted_df, 'Year')
        melted_df = convert_to_numeric(melted_df, f"{level}_difference")

        # Get the top 3 countries with the highest difference in absolute percentage
        top_countries = melted_df[melted_df['Year'] == year].sort_values(by = f"{level}_difference", key = lambda x: x.abs(), ascending = False).head(3)[['Country', f"{level}_difference"]]

        for i, (country, percentage) in enumerate(top_countries.itertuples(index = False, name = None), start = 1):
            print(f"The {ordinal(i)} country with the highest difference in percentage between M and F for {level} education is {country} with a percentage of {percentage:.2f}%")
            valueM = level_datasets[level]['M'].loc[ level_datasets[level]['M']['Country'] == country, str(year) ].values[0]
            valueF = level_datasets[level]['F'].loc[ level_datasets[level]['F']['Country'] == country, str(year) ].values[0]
            donut_data.append({'name': country, 'abbreviation': country_abbreviations.get(country),'level': level, 'rank': i, 'valueM': valueM, 'valueF': valueF})

    donut_df = pd.DataFrame(donut_data)
    donut_df.to_csv(f"donut{year}.csv", index = False)


In the year 2009:
The 1st country with the highest difference in percentage between M and F for low education is Spain with a percentage of 14.90%
The 2nd country with the highest difference in percentage between M and F for low education is Greece with a percentage of 12.12%
The 3rd country with the highest difference in percentage between M and F for low education is Malta with a percentage of 11.71%
The 1st country with the highest difference in percentage between M and F for medium education is Bulgaria with a percentage of 12.53%
The 2nd country with the highest difference in percentage between M and F for medium education is Türkiye with a percentage of 11.79%
The 3rd country with the highest difference in percentage between M and F for medium education is North Macedonia with a percentage of 11.33%
The 1st country with the highest difference in percentage between M and F for high education is Croatia with a percentage of -33.95%
The 2nd country with the highest difference in pe

For each year (from 2009 to 2022) we retrieve the top 3 countries with highest percentual difference between M and F (for low, medium and high education level), considering THE SIGN of this percentage (NOT considering the absolute value).

In [23]:
for year in range(2009, 2023):
    print(f"\nIn the year {year}:")

    for level in levels:
        percentage_df = percentage_datasets[f"{level}_percentage"]

        # Reshape data using melt
        melted_df = pd.melt(percentage_df, id_vars=['Country'], var_name='Year', value_name=f"{level}_difference")

        # Convert 'Year' and difference columns to numeric
        melted_df = convert_to_numeric(melted_df, 'Year')
        melted_df = convert_to_numeric(melted_df, f"{level}_difference")

        # Get the top 3 countries with the highest difference in percentage
        top_countries = melted_df[melted_df['Year'] == year].nlargest(3, f"{level}_difference")[['Country', f"{level}_difference"]]

        # Print information for each country
        for i, (country, percentage) in enumerate(top_countries.itertuples(index = False, name = None), start=1):
            print(f"The {ordinal(i)} country with the highest difference in percentage between M and F for {level} education is {country} with a percentage of {percentage:.2f}%")


In the year 2009:
The 1st country with the highest difference in percentage between M and F for low education is Spain with a percentage of 14.90%
The 2nd country with the highest difference in percentage between M and F for low education is Greece with a percentage of 12.12%
The 3rd country with the highest difference in percentage between M and F for low education is Malta with a percentage of 11.71%
The 1st country with the highest difference in percentage between M and F for medium education is Bulgaria with a percentage of 12.53%
The 2nd country with the highest difference in percentage between M and F for medium education is Türkiye with a percentage of 11.79%
The 3rd country with the highest difference in percentage between M and F for medium education is North Macedonia with a percentage of 11.33%
The 1st country with the highest difference in percentage between M and F for high education is Switzerland with a percentage of 2.22%
The 2nd country with the highest difference in 

## Education level: percentage --> we are NOT using this dataset for now!

We import the 18 sheets of the *pop_edu_sex_age_percentage* excel file (skipping useless rows/columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *education_per_sheets*.

In [37]:
sheet_names = [f'Sheet {i}' for i in range(1, 19)]
file_path = '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_percentage.xlsx'
rows_to_skip = list(range(0, 10)) + [11, 12, 13, 44] + list(range(49, 56))
selected_columns = ['TIME'] + [str(year) for year in years]

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'education_per_sheet1', ..., 'education_per_sheet36'
    education_per_sheets = {f'education_per_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                            for i, sheet_name in enumerate(sheet_names, start = 1)}

We add a column *Abbreviation* to the 18 datasets.

In [ ]:
for i in range(1, 19):
    education_per_sheets[f'education_per_sheet{i}'].rename(columns = {'TIME': 'Country'}, inplace = True)
    education_per_sheets[f'education_per_sheet{i}']['Abbreviation'] = education_per_sheets[f'education_per_sheet{i}']['Country'].map(country_abbreviations)

## Enrolment by age --> still to explore!

Grafico 3 : solo per alcuni paesi (solo totale, no maschi/femmine)

full and part-time

età sull'asse x,

numero sulle y,

stacked (o streamgraph)

colori diversi per categorie diverse

In [ ]:
enrolment = pd.read_csv("/content/drive/MyDrive/DV_project/story2/enrolment_by_age.csv")

In [ ]:
enrolment

,COUNTRY,Country,SEX,Gender,AGE,Age,INTENSITY,Intensity,EDUCATION_LEV,Education level,YEAR,Year,Value,Flag Codes,Flags
0,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2013,2013,NaN,m,Missing value; data cannot exist
1,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2014,2014,NaN,m,Missing value; data cannot exist
2,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2015,2015,NaN,m,Missing value; data cannot exist
3,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2016,2016,NaN,m,Missing value; data cannot exist
4,AUT,Austria,F,Female,Y15,15 years,PT,Part-time,ISCED11_0,Early childhood education,2017,2017,NaN,m,Missing value; data cannot exist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524155,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2017,2017,55553.0,NaN,NaN
524156,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2018,2018,57230.0,NaN,NaN
524157,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2019,2019,48242.0,NaN,NaN
524158,BGR,Bulgaria,M,Male,_T,Total,PT,Part-time,ISCED11_T,Total,2020,2020,42941.0,NaN,NaN


## Early leavers --> still to explore!

We import the 15 sheets of the *early_leavers_by_sex_labour* excel file (skipping useless rows/columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *early_leavers_sheets*.

In [46]:
sheet_names = [f'Sheet {i}' for i in range(1, 16)]
file_path = '/content/drive/MyDrive/DV_project/story2/early_leavers_by_sex_labour.xlsx'
rows_to_skip = list(range(0, 10)) + [11, 13, 44] + list(range(49, 58))
selected_columns = ['TIME'] + [str(year) for year in years]

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'early_leavers_sheet1', ..., 'early_leavers_sheet15'
    early_leavers_sheets = {f'early_leavers_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                            for i, sheet_name in enumerate(sheet_names, start = 1)}

We add a column *Abbreviation* to the 15 datasets.

In [47]:
for i in range(1, 16):
    early_leavers_sheets[f'early_leavers_sheet{i}'].rename(columns = {'TIME': 'Country'}, inplace = True)
    early_leavers_sheets[f'early_leavers_sheet{i}']['Abbreviation'] = early_leavers_sheets[f'early_leavers_sheet{i}']['Country'].map(country_abbreviations)

Grafico 4 : n. early leavers senza distinzioni (ne per genere ne per occupazione) per ogni paese e , lollipop chart + dato complessivo UE

In [50]:
early_leavers_datasets = {}

for year in years:
    early_leavers_datasets[f'early_leavers_{year}'] = early_leavers_sheets['early_leavers_sheet1'][['Country', str(year), 'Abbreviation']].copy()
    early_leavers_datasets[f'early_leavers_{year}'].dropna(subset = [str(year)], inplace = True)
    early_leavers_datasets[f'early_leavers_{year}'].sort_values(by = [str(year)], ascending = [False], inplace = True)

In [54]:
header = ['name', 'abundance', 'abbreviation']

for year in years:
  csv_name = f'lollipop{year}.csv'
  early_leavers_datasets[f'early_leavers_{year}'].to_csv(csv_name, index = False, header = header)

  # csv_name = f'lollipop{year}_EU.csv'
  # eu_index = early_leavers_datasets[f'early_leavers_{year}'][early_leavers_datasets[f'early_leavers_{year}']['Abbreviation'] == 'EU'].index[0]
  # early_leavers_datasets[f'early_leavers_{year}'] = pd.concat([early_leavers_datasets[f'early_leavers_{year}'].loc[eu_index:],
  #                                                              early_leavers_datasets[f'early_leavers_{year}'].loc[:eu_index - 1]], ignore_index = True)
  # early_leavers_datasets[f'early_leavers_{year}'].to_csv(csv_name, index = False, header = header)

Grafico 5 : visualizzare early leavers, piramide con categorie (cercano, non cercano, etc.) a livello complessivo, NO valori per stato e per anno

# Download all CSV files

We download all the CSV files we created.

In [55]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname = os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>